In [1]:
import numpy as np
import torch
import torchvision as tv
import matplotlib.pyplot as plt
from torch import nn
import time

In [2]:
# 1.Загрузка данных
data_train = tv.datasets.FashionMNIST('D:\НЕТОЛОГИЯ\Deep ML\файлы', download = False, train = True, transform = tv.transforms.ToTensor())
data_test = tv.datasets.FashionMNIST('D:\НЕТОЛОГИЯ\Deep ML\файлы', download = False, train = False, transform = tv.transforms.ToTensor())

In [3]:
print('число классов:', len(data_train.classes))
print()
print('train:', data_train.data.shape, data_train)
print()
print('test:', data_test.data.shape, data_test)

число классов: 10

train: torch.Size([60000, 28, 28]) Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: D:\НЕТОЛОГИЯ\Deep ML\файлы
    Split: Train
    StandardTransform
Transform: ToTensor()

test: torch.Size([10000, 28, 28]) Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: D:\НЕТОЛОГИЯ\Deep ML\файлы
    Split: Test
    StandardTransform
Transform: ToTensor()


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# Данные для модели
batch_size = 512
input_shape = data_train.data.shape[1] * data_train.data.shape[2]
class_number = len(data_train.classes)

train_load = torch.utils.data.DataLoader(data_train, batch_size = batch_size)
test_load = torch.utils.data.DataLoader(data_test, batch_size = batch_size)

In [6]:
#Слои
model = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=3, padding=0),
    nn.Tanh(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(6, 16, kernel_size=3),
    nn.Tanh(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(16, 120, kernel_size=3),
    nn.Flatten(),
    nn.Linear(120, 84),
    nn.Tanh(),
    nn.Linear(84, 10)
)
model

Sequential(
  (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (1): Tanh()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (4): Tanh()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(16, 120, kernel_size=(3, 3), stride=(1, 1))
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=120, out_features=84, bias=True)
  (9): Tanh()
  (10): Linear(in_features=84, out_features=10, bias=True)
)

In [7]:
# Создание модели. Функция для создания модели

def model_create(model, optim, learn_rate):
    model = model
    
    model = model.to(device)
    loss = torch.nn.CrossEntropyLoss()
    if optim == 'SGD': optimizer = torch.optim.SGD(model.parameters(), lr = learn_rate)
    elif optim == 'Adam': optimizer = torch.optim.Adam(model.parameters(), lr = learn_rate)
    elif optim == 'RMSprop': optimizer = torch.optim.RMSprop(model.parameters(), lr = learn_rate)
        
    return model, loss, optimizer

In [12]:
m1, loss1, sgd_01 = model_create(model, optim = 'SGD', learn_rate = 0.01)

In [9]:
# Создание модели. Функция для обучения и расчета точности на тесте
def model_acc(num_epochs, model, loss, optimizer, test_loader, train_loader):
    for epoch in range(num_epochs):
        train_iters, train_passed = 0., 0.
        train_loss, train_acc = 0., 0.
        start = time.time()
        
        # Обучение
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            l = loss(y_pred, y)
            l.backward()
            optimizer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim = 1) == y).sum().item()
            train_iters += 1
            train_passed += len(x)
    
        print(f'ep: {epoch}, loss: {train_loss/train_iters:.4f}, accuracy: {train_acc/train_passed:.4f}, time {time.time() - start:.1f} sec')

    # Тест
    test_iters, test_passed  = 0., 0.
    test_loss, test_acc = 0., 0.
    
    model.eval()
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        l = loss(y_pred, y)

        test_loss += l.item()
        test_acc += (y_pred.argmax(dim = 1) == y).sum().item()
        test_iters += 1
        test_passed += len(x)
    print()
    print(f'test accuracy: {test_acc/test_passed}')

In [14]:
model_acc(num_epochs = 10, 
          model = m1, 
          loss = loss1, 
          optimizer = sgd_01, 
          test_loader = test_load, 
          train_loader = train_load
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x1080 and 120x84)